# Localisation et détection d'objet

Dans ce TP, nous allons mettre en pratique certaines des méthodes présentées en cours pour localiser des objets dans une image.

En localisation et détection, on cherche à déterminer la position d'un objet, ainsi que sa classe, sous la forme d'une boîte englobante de largeur $b_w$ et hauteur $b_h$, et dont le centre a pour coordonnées le point $(b_x, b_y)$.

<center> <img src="https://drive.google.com/uc?id=1_jHHv6ZDe-3Xz25jIZ6o177laBEfmMRR" style="width:500;height:300px;"></center>
<caption><center> Figure 1: Modèle de boîte englobante utilisé pour la localisation </center></caption>

Le problème de localisation considère qu'un seul objet est présent sur l'image, alors que le problème de détection cherche à déterminer l'ensemble des objets présents sur l'image.



Pour commencer, récupérez les images de la base de données :


In [1]:
!git clone https://github.com/axelcarlier/mangeoires_loc.git

Cloning into 'mangeoires_loc'...
Updating files:  54% (1264/2330)
Updating files:  55% (1282/2330)
Updating files:  56% (1305/2330)
Updating files:  57% (1329/2330)
Updating files:  58% (1352/2330)
Updating files:  59% (1375/2330)
Updating files:  60% (1398/2330)
Updating files:  61% (1422/2330)
Updating files:  62% (1445/2330)
Updating files:  63% (1468/2330)
Updating files:  64% (1492/2330)
Updating files:  65% (1515/2330)
Updating files:  66% (1538/2330)
Updating files:  67% (1562/2330)
Updating files:  68% (1585/2330)
Updating files:  69% (1608/2330)
Updating files:  70% (1631/2330)
Updating files:  71% (1655/2330)
Updating files:  72% (1678/2330)
Updating files:  73% (1701/2330)
Updating files:  74% (1725/2330)
Updating files:  75% (1748/2330)
Updating files:  76% (1771/2330)
Updating files:  77% (1795/2330)
Updating files:  78% (1818/2330)
Updating files:  79% (1841/2330)
Updating files:  80% (1864/2330)
Updating files:  81% (1888/2330)
Updating files:  82% (1911/2330)
Updating f

La base de données consiste en des photographies prises par une caméra reliée à une Raspberry Pi, cachée dans une mangeoire. Plusieurs mangeoires sont disséminées dans la nature en Occitanie, et l'objectif de [ce projet](https://econect.cnrs.fr/) est la reconnaissance des espèces et le comptage des individus qui viennent se poser devant le caméra, afin de suivre l'évolution des populations d'oiseaux et ainsi monitorer la biodiversité.

La base de données qui vous est fournie regroupe 11 espèces d'animaux, majoritairement des oiseaux, désignés par un code :

1. Mésange charbonnière (**MESCHA**)
2. Verdier d'Europe (**VEREUR**)
3. Écureuil roux (**ECUROU**)
4. Pie bavarde (**PIEBAV**)
5. Sittelle torchepot (**SITTOR**)
6. Pinson des arbres (**PINARB**)
7. Mésange noire (**MESNOI**)
8. Mésange nonnette (**MESNON**)
9. Mésange bleue (**MESBLE**)
10. Rouge-gorge (**ROUGOR**)
11. Accenteur mouchet (**ACCMOU**)



<center> <img src="https://drive.google.com/uc?id=1Eit86N4D0Ea7ai1rBa0o-GmTwckhP9i0" width=200>
<img src="https://drive.google.com/uc?id=1lC7WL93UqDT_KV2m21yx99N5dkf98nCY" width=200>
<img src="https://drive.google.com/uc?id=10tzJORcSrSckDWmBDBtc8FUirxdbRxri" width=200>
<img src="https://drive.google.com/uc?id=1IHZp_B6bc8bADtAReRHhcyOjDQZAXDBQ" width=200></center>
<caption><center> Figure 2: Exemples d'images de la base de données </center></caption>

# Localisation et classification d'objet

Dans cette partie, nous allons nous concentrer sur le problème de la localisation d'un seul objet par classe. La base de données a été épurée pour se concentrer uniquement sur ce cas.


In [ ]:
import PIL
from PIL import Image
import csv
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam


## Préparation des données

Le code ci-dessous permet de charger les données et les formater pour la classification. Prenez le temps de regarder un peu le format des labels $y$.
Notez que les images sont rendues carrées lors du chargement.

In [ ]:
# Lecture du CSV contenant les informations relatives à la base de données
dataset = []
with open('mangeoires_loc/bd_mangeoires_equilibre.csv', newline='') as csvfile:
	filereader = csv.reader(csvfile, delimiter=' ', quotechar='|')
	for row in filereader:
		data = row[0].split(',')
		if data[0] != 'Data':
			box = [float(data[5]), float(data[6]), float(data[7]), float(data[8])]
			new_entry = {'type': data[0], 'specie': data[1], 'path': data[2], 'shape': [float(data[3]), float(data[4])], 'box': box}
			dataset.append(new_entry)

# Nombre de classes de la base de données et intitulé des classes
class_labels = list(dict.fromkeys([item['specie'] for item in dataset]))
num_classes = len(class_labels)

# Extraction des données d'apprentissage et de test
dataset_train = [item for item in dataset if item['type']=='TRAIN']
dataset_test = [item for item in dataset if item['type']=='TEST']

print(class_labels)

In [ ]:
def build_localization_tensors(image_size, dataset, num_classes):
  # Préparation des structures de données pour x et y
  x = np.zeros((len(dataset), image_size, image_size, 3))
  y = np.empty((len(dataset), num_classes + 5)) # 1 + 4 + num_classes : présence / boîte englobante / classes

  # Compteur de parcours du dataset
  i = 0

  for item in dataset:
    # Lecture de l'image
    img = Image.open('mangeoires_loc/' + item['path'])
    # Mise à l'échelle de l'image
    img = img.resize((image_size,image_size), Image.ANTIALIAS)
    # Remplissage de la variable x
    x[i] = np.asarray(img)

    y[i, 0] = 1 # Un objet est toujours présent !

    # Coordonnées de boîte englobante
    img_shape = item['shape']
    box = item['box']
    bx = (box[0] + (box[2] - box[0])/2)/img_shape[0]
    by = (box[1] + (box[3] - box[1])/2)/img_shape[1]
    bw = (box[2] - box[0])/img_shape[0]
    bh = (box[3] - box[1])/img_shape[1]
    y[i, 1] = bx
    y[i, 2] = by
    y[i, 3] = bw
    y[i, 4] = bh

    # Probabilités de classe, sous la forme d'une one-hot vector
    label = class_labels.index(item['specie'])
    classes_probabilities = keras.utils.to_categorical(label, num_classes=num_classes)
    y[i, 5:] = classes_probabilities

    i = i+1

  return x, y


Séparation des données d'entraînement pour extraire un ensemble de validation, et pré-traitement des données.

In [ ]:
from sklearn.model_selection import train_test_split


# Pour la suite du TP on considèrera des images de taille 64x64x3
# Augmenter cette valeur donnerait de meilleurs résultats mais nécessiterait des calculs plus long.
IMAGE_SIZE = 64

# Lecture des données d'entraînement et de test
x, y = build_localization_tensors(IMAGE_SIZE, dataset_train, num_classes)
x_test, y_test = build_localization_tensors(IMAGE_SIZE, dataset_test, num_classes)

#Extraction d'un ensemble de validation
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.10, random_state=42)

# Pour améliorer l'entraînement, on peut centrer-réduire les coordonnées des bounding boxes...
y_std = np.std(y_train, axis=0)
y_mean = np.mean(y_train, axis=0)
y_train[...,1:5] = (y_train[...,1:5] - y_mean[1:5])/y_std[1:5]
y_val[...,1:5] = (y_val[...,1:5] - y_mean[1:5])/y_std[1:5]
y_test[...,1:5] = (y_test[...,1:5] - y_mean[1:5])/y_std[1:5]

# ... et normaliser les valeurs de couleur
x_train = x_train/255
x_val = x_val/255
x_test = x_test/255

## Fonctions utiles

Une fonction de calcul de l'intersection sur union, qui nous sera utile pour les métriques d'évaluation de nos méthodes :

$$ IoU (R_1, R_2) = \frac{\mathcal{A} (R_1 \cap R_2)}{\mathcal{A} (R_1 \cup R_2)} = \frac{\mathcal{A} (R_1 \cap R_2)}{\mathcal{A} (R_1) + \mathcal{A} (R_2) -  \mathcal{A} (R_1 \cap R_2)} $$

<center>
<img src="https://drive.google.com/uc?id=1BQx2kDOCltZ_5gcnKVWTVUVNmGk-7qBC" width=500>
</center>


In [ ]:
### A COMPLETER
def intersection_sur_union(box1, box2):
  """
  Calcul de l'intersection sur union entre deux rectangles box1 et box2

  Arguments:
  box1, box2 -- les coordonnées des deux rectangles, chacun sous la forme [cx, cy, w, h]
                où (cx, cy) désigne les coordonnées du centre du rectangle,
                w sa largeur et h sa hauteur
  
  
  Retourne :
  iou -- la valeur d'intersection sur union entre les deux rectangles
  """
  
  
  x1 = max(box1[0] - box1[2]/2, box2[0] - box2[2]/2)
  y1 = max(box1[1] - box1[3]/2, box2[1] - box2[3]/2)
  x2 = min(box1[0] + box1[2]/2, box2[0] + box2[2]/2)
  y2 = min(box1[1] + box1[3]/2, box2[1] + box2[3]/2)



In [ ]:
print(intersection_sur_union([2.5, 2, 1, 4], [2, 3, 4, 2]))  # Résultat attendu : 0.2
print(intersection_sur_union([2.5, 2, 1, 4], [5, 3, 4, 2]))  # Résultat attendu : 0.0

Calcul des différentes métriques :

$$ P = \frac{TP}{TP + FP} $$

$$ R = \frac{TP}{TP + FN} $$

$$ F1 = \frac{2 * P * R}{P + R} $$

où $TP$ désigne le nombre de vrais positifs, $FP$ le nombre de faux positifs, $FN$ le nombre de faux négatifs, $P$ la précision, $R$ le rappel et $F1$ le F1-score.

On considère souvent qu'une détection est correcte si la classification est valide et que l'intersection sur union entre vérité terrain et prédiction est supérieure à 0.5 (on utilisera un seuil modifiable *iou_thres*).

In [ ]:
# A COMPLETER
def global_accuracy(y_true, y_pred, iou_thres=0.5):
  """
  Calcul, pour chaque classe de la précision, du rappel et du F1-score ainsi
  que du pourcentage global de bonnes détections.

  Arguments:
  y_true -- les labels de la vérité terrain, de dimension (M, 1+4+N) où M désigne
          le nombre d'éléments du dataset et N le nombre de classes (11 dans notre cas)
  y_pred -- les labels prédits par un modèle, de dimension (M, 1+4+N)
  iou_thres -- seuil d'intersection sur union entre une boîte "vérité-terrain" et
              une boite prédite au-dessus duquel on considère que la prédiction est correcte

  Retourne :
  class_res -- liste de longueur N contenant des dictionnaires sous la forme
            {"Précision": p, "Rappel": r, "F-score": f} résumant les métriques
            précision, rappel et F1-score pour chacune des classes.
  accuracy -- pourcentage global de bonnes détections
  """
  # Initialisation des métriques : nombre de vrais positifs (TP), faux positifs (FP)
  # et faux négatifs (FN) pour chaque classe
  class_metrics = []
  for i in range(num_classes):
    class_metrics.append({'TP': 0, 'FP': 0, 'FN': 0})

  # Nombres de détections correctes et de détections incorrectes
  total_correct_detections = 0
  total_incorrect_detections = 0
  for i in range(y_true.shape[0]):
    # Labels vérité-terrain et prédits
    groundtruth_label = np.argmax(y_true[i,5:])
    predicted_label = np.argmax(y_pred[i,5:])

    # Coordonnées de boîtes englobantes réelles et prédites
    bx_true = (y_true[i,1]*y_std[1] + y_mean[1])
    by_true = (y_true[i,2]*y_std[2] + y_mean[2])
    bw_true = (y_true[i,3]*y_std[3] + y_mean[3])
    bh_true = (y_true[i,4]*y_std[4] + y_mean[4])
    bx_pred = (y_pred[i,1]*y_std[1] + y_mean[1])
    by_pred = (y_pred[i,2]*y_std[2] + y_mean[2])
    bw_pred = (y_pred[i,3]*y_std[3] + y_mean[3])
    bh_pred = (y_pred[i,4]*y_std[4] + y_mean[4])

    # Calcul de l'intersection sur union
    iou = intersection_sur_union([bx_true, by_true, bw_true, bh_true], [bx_pred, by_pred, bw_pred, bh_pred])

    ### A COMPLETER
    # Si la détection est correcte :
    if groundtruth_label == predicted_label and iou > iou_thres:
      
      # Mise à jour des vrais positifs
      class_metrics[groundtruth_label]['TP'] += 1
      total_correct_detections += 1
    else:
      # Mise à jour des faux positifs et des faux négatifs
      class_metrics[groundtruth_label]['FN'] += 1
      class_metrics[predicted_label]['FP'] += 1
      total_incorrect_detections += 1
      

  class_res = []
  for i in range(num_classes):
    P = class_metrics[i]['TP'] / (class_metrics[i]['TP'] + class_metrics[i]['FP'])
    R = ...
    F_score = ...
    class_res.append({'Precision': P, 'Rappel': R, 'F-score': F_score})

  accuracy = 
  ### FIN
  return class_res, accuracy

In [ ]:
num_class_test = 3
class_labels_test = ['Classe 1', 'Classe 2', 'Classe 3']
y_true_test = np.ones((num_class_test,8))
y_true_test[0,:2] = [0.5, 0.5]
y_true_test[0, 5:] = [1, 0, 0]
y_true_test[1,:2] = [0.5, 0.5]
y_true_test[1, 5:] = [0, 1, 0]
y_true_test[2,:2] = [0.5, 0.5]
y_true_test[2, 5:] = [0, 0, 1]
y_pred_test = np.ones((num_class_test,8))
y_pred_test[0,:2] = [0.6, 0.6]
y_pred_test[0, 5:] = [1, 0, 0]
y_pred_test[1,:2] = [2.5, 2.5]
y_pred_test[1, 5:] = [0, 0, 1]
y_pred_test[2,:2] = [0.6, 0.6]
y_pred_test[2, 5:] = [0, 0, 1]

class_res_test, acc_test = global_accuracy(y_true_test, y_pred_test)

print(f"La précision globale est de {acc_test:.1f}%")

print()
print("--------------------------------------------")
print("|  Classe  | Précision | Rappel | F1-score |")
print("--------------------------------------------")
for i in range(num_class_test):
  print(f"| {class_labels_test[i]:9s}|   {class_res_test[i]['Precision']:.2f}    |  {class_res_test[i]['Rappel']:.2f}  |   {class_res_test[i]['F-score']:.2f}   |")
  print("--------------------------------------------")

**Affichage attendu :**
```
La précision globale est de 66.7%

--------------------------------------------
|  Classe  | Précision | Rappel | F1-score |
--------------------------------------------
| Classe 1 |   1.00    |  1.00  |   1.00   |
--------------------------------------------
| Classe 2 |   0.00    |  0.00  |   0.00   |
--------------------------------------------
| Classe 3 |   0.50    |  1.00  |   0.67   |
--------------------------------------------
```

La fonction ci-dessous permet de calculer l'intersection sur union  sur des tenseurs (et non des tableaux numpy), elle sera donc utilisable comme métrique pendant l'entraînement.

In [ ]:
def compute_iou(y_true, y_pred):
  ### "Dénormalisation" des coordonnées des boîtes englobantes
  pred_box_xy = y_pred[..., 0:2]* y_std[0:2] + y_mean[0:2]
  true_box_xy = y_true[..., 0:2]* y_std[0:2] + y_mean[0:2]

  ### "Dénormalisation" des largeur et hauteur des boîtes englobantes
  pred_box_wh = y_pred[..., 2:4] * y_std[2:4] + y_mean[2:4]
  true_box_wh = y_true[..., 2:4] * y_std[2:4] + y_mean[2:4]

  # Calcul des coordonnées minimales et maximales des boiptes englobantes réelles
  true_wh_half = true_box_wh / 2.
  true_mins    = true_box_xy - true_wh_half
  true_maxes   = true_box_xy + true_wh_half

  # Calcul des coordonnées minimales et maximales des boiptes englobantes prédites
  pred_wh_half = pred_box_wh / 2.
  pred_mins    = pred_box_xy - pred_wh_half
  pred_maxes   = pred_box_xy + pred_wh_half

  # Détermination de l'intersection des boîtes englobantes
  intersect_mins  = tf.maximum(pred_mins,  true_mins)
  intersect_maxes = tf.minimum(pred_maxes, true_maxes)
  intersect_wh    = tf.maximum(intersect_maxes - intersect_mins, 0.)
  intersect_areas = intersect_wh[..., 0] * intersect_wh[..., 1]

  # Aire des boîtes englobantes prédites et réelles
  true_areas = true_box_wh[..., 0] * true_box_wh[..., 1]
  pred_areas = pred_box_wh[..., 0] * pred_box_wh[..., 1]

  # Aire de l'union des boîtes prédites et réelles
  union_areas = pred_areas + true_areas - intersect_areas

  iou_scores  = tf.truediv(intersect_areas, union_areas)
  return iou_scores

def iou():
  def iou_metrics(y_true, y_pred):
    return compute_iou(y_true, y_pred)
  iou_metrics.__name__= "IoU"
  return iou_metrics

Visualisation des données et labels

In [ ]:
# Si seuls x et y sont indiqués, on tire au hasard un numéro d'image et on affiche le label y associé  à l'image
# Si un 2e y, nommé y_pred, est indiqué, alors les deux labels sont affichés côte à côte, afin de pouvoir les comparer
# Enfin on peut également indiquer l'id de l'image que l'on souhaite visualiser.
def print_data_localisation(x, y, y_pred=[], id=None, image_size=64):
  if id==None:
    # Tirage aléatoire d'une image dans la base
    num_img = np.random.randint(x.shape[0]-1)
  else:
    num_img = id

  img = x[num_img]
  lab = y[num_img]

  colors = ["blue", "yellow", "red", "orange", "coral", "gold", "ivory", "fuchsia", "purple", "cyan", "navy"] # Différentes couleurs pour les différentes classes
  classes = ['MESCHA', 'VEREUR', 'ECUROU', 'PIEBAV', 'SITTOR', 'PINARB', 'MESNOI', 'MESNON', 'MESBLE', 'ROUGOR', 'ACCMOU']

  if np.any(y_pred):
    plt.subplot(1, 2, 1)

  # Affichage de l'image
  plt.imshow(img)
  # Détermination de la classe
  class_id = np.argmax(lab[5:])

  # Détermination des coordonnées de la boîte englobante dans le repère image
  ax = (lab[1]*y_std[1] + y_mean[1]) * image_size
  ay = (lab[2]*y_std[2] + y_mean[2]) * image_size
  width = (lab[3]*y_std[3] + y_mean[3]) * image_size
  height = (lab[4]*y_std[4] + y_mean[4]) * image_size
  #print("x: {}, y: {}, w: {}, h:{}".format(ax,ay,width, height))
  # Détermination des extrema de la boîte englobante
  p_x = [ax-width/2, ax+width/2]
  p_y = [ay-height/2, ay+height/2]
  # Affichage de la boîte englobante, dans la bonne couleur
  plt.plot([p_x[0], p_x[0]],p_y,color=colors[class_id])
  plt.plot([p_x[1], p_x[1]],p_y,color=colors[class_id])
  plt.plot(p_x,[p_y[0],p_y[0]],color=colors[class_id])
  plt.plot(p_x,[p_y[1],p_y[1]],color=colors[class_id])
  plt.title("Vérité Terrain : Image {} - {}".format(num_img, classes[class_id]))

  if np.any(y_pred):
    plt.subplot(1, 2, 2)
    # Affichage de l'image
    plt.imshow(img)
    lab = y_pred[num_img]
    # Détermination de la classe
    class_id = np.argmax(lab[5:])

    # Détermination des coordonnées de la boîte englobante dans le repère image
    ax = (lab[1]*y_std[1] + y_mean[1]) * image_size
    ay = (lab[2]*y_std[2] + y_mean[2]) * image_size
    width = (lab[3]*y_std[3] + y_mean[3]) * image_size
    height = (lab[4]*y_std[4] + y_mean[4]) * image_size
    #print("x: {}, y: {}, w: {}, h:{}".format(ax,ay,width, height))
    # Détermination des extrema de la boîte englobante
    p_x = [ax-width/2, ax+width/2]
    p_y = [ay-height/2, ay+height/2]
    # Affichage de la boîte englobante, dans la bonne couleur
    plt.plot([p_x[0], p_x[0]],p_y,color=colors[class_id])
    plt.plot([p_x[1], p_x[1]],p_y,color=colors[class_id])
    plt.plot(p_x,[p_y[0],p_y[0]],color=colors[class_id])
    plt.plot(p_x,[p_y[1],p_y[1]],color=colors[class_id])
    plt.title("Prédiction : Image {} - {}".format(num_img, classes[class_id]))

  plt.show()

for i in range(10):#x.shape[0]):
    print_data_localisation(x_train, y_train, image_size=IMAGE_SIZE, id=i)


Fonction d'affichage des courbes d'apprentissage et de validation

In [ ]:
def plot_training_analysis(history, metric='loss'):

  loss = history.history[metric]
  val_loss = history.history['val_' + metric]

  epochs = range(len(loss))

  plt.plot(epochs, loss, 'b', linestyle="--",label='Training ' + metric)
  plt.plot(epochs, val_loss, 'g', label='Validation ' + metric)
  plt.title('Training and validation ' + metric)
  plt.legend()

  plt.show()

## Travail à faire



<center> <img src="https://drive.google.com/uc?id=1YzCZe4pgnjJDVGklAaCHZ7HhlPJsadg9" width=500></center>
<caption><center> Figure 3: Illustration de l'architecture du réseau à construire.  </center></caption>

Complétez les codes qui vous sont fournis pour obtenir un algorithme de localisation.
Vous pouvez utiliser n'importe quelle base convolutive de votre choix (**commencez par exemple par réutiliser celle que vous avez implémenté au TP3**), en revanche vous devrez porter une attention particulière à la couche de sortie.

Vous allez en fait produire 3 sorties différentes : une caractérisant la présence d'un objet, une autre fournissant les coordonnées de la boîte englobante, et enfin une dernière effectuant la classification.

In [ ]:
def create_model_localisation(input_shape=(64, 64, 3)):

  input_layer = Input(shape=input_shape)


  conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
  
  pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
  
  conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
  
  pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
  
  conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
  
  pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
  
  conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
  
  pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
  # Convolutions, Pooling, Dense,  à vous de former votre réseau
  
  # Votre dernière couche doit mettre à jour une variable prev_layer, réutilisée dans les couches de sortie ci-dessous
  prev_layer = Flatten()(pool4)
   
  # Sortie caractérisant la présence d'un objet
  output_p = Dense(1, activation='sigmoid', name='presence')(prev_layer)
  # Sortie caractérisant les coordonnées de boîte englobante
  output_coord = Dense(4,activation = 'linear', name='bounding_box')(prev_layer)
  
  # Sortie caractérisant les probabilités de classe
  output_class = Dense(num_classes, activation='softmax', name='classes')(prev_layer)

  output= [output_p, output_coord, output_class]
  model = Model(input_layer, output)

  return model

<center> <img src="https://drive.google.com/uc?id=1bnh8zU7Os-w-5TT8hV4xDoThKQc-Ywc2" width=500></center>
<caption><center> Figure 4: Illustration des fonctions de coût à utiliser pour l'entraînement. </center></caption>

Pour entraîner votre réseau, vous allez donc devoir associer une fonction de coût à chacune des sorties du réseau. La fonction de coût totale sera la somme des trois fonctions de coût précédemment définies, pondérées par des poids définis dans la variable *loss_weights*.

**Prenez le temps de tester différentes valeurs de *loss_weights* en fonction de l'évolution des métriques que vous observerez pendant l'entraînement.**

Vous évaluerez vos résultats de manière qualitative (en affichant les boîtes englobantes prédites et réelles) mais aussi quantitatives grâce aux fonctions définies dans la section précédente. **N'hésitez pas à modifier un peu le paramètre iou_threshold positionné par défaut à 0.5** (une valeur de 0.4 vous permettra d'obtenir de meilleurs résultats !)

In [ ]:
batch_size=16
model = create_model_localisation()
opt = Adam(learning_rate=3e-4)

# Ici mettre, dans l'ordre, les fonctions de coût associées à chacune des sorties
loss=['binary_crossentropy', 'mean_squared_error', 'categorical_crossentropy']
# On va associer une métrique à chaque sortie : l'accuracy pour les deux classifications,
# et l'IoU définie plus tôt pour la qualité des boîtes englobantes.
metrics=[ ['accuracy'], [iou()], ['accuracy']]
loss_weights = [1, 1, 1]

model.compile(loss=loss,
              optimizer=opt,
              metrics=metrics,
              loss_weights=loss_weights
              )
history = model.fit(x_train, [y_train[:,0], y_train[:,1:5], y_train[:,5:]],
              epochs=50,
              batch_size=batch_size,
              validation_data=(x_val, [y_val[:,0], y_val[:,1:5], y_val[:,5:]]))



In [ ]:
# Analyse des résultats : courbes d'évolution de la fonction de perte, et de l'IoU des boîtes englobantes ainsi que de la précision des classes prédites
plot_training_analysis(history, metric='loss')
plot_training_analysis(history, metric='coord_IoU')
plot_training_analysis(history, metric='classes_accuracy')

# Prédiction des données de test
y_pred_presence, y_pred_coords, y_pred_classes = model.predict(x_test)
y_pred = np.zeros(y_test.shape)
for i in range(y_pred.shape[0]):
  y_pred[i, 0] = y_pred_presence[i]
  y_pred[i, 1:5] = y_pred_coords[i]
  y_pred[i, 5:] = y_pred_classes[i]

# Affichage des résultats sur plusieurs images
print_data_localisation(x_test, y_test, y_pred = y_pred, id=1, image_size=IMAGE_SIZE)
print_data_localisation(x_test, y_test, y_pred = y_pred, id=2, image_size=IMAGE_SIZE)
print_data_localisation(x_test, y_test, y_pred = y_pred, id=3, image_size=IMAGE_SIZE)
print_data_localisation(x_test, y_test, y_pred = y_pred, id=4, image_size=IMAGE_SIZE)
print_data_localisation(x_test, y_test, y_pred = y_pred, id=5, image_size=IMAGE_SIZE)
print_data_localisation(x_test, y_test, y_pred = y_pred, id=6, image_size=IMAGE_SIZE)
print_data_localisation(x_test, y_test, y_pred = y_pred, id=7, image_size=IMAGE_SIZE)
print_data_localisation(x_test, y_test, y_pred = y_pred, id=8, image_size=IMAGE_SIZE)

In [ ]:
class_res, accuracy = global_accuracy(y_test, y_pred)

print(f"La précision globale est de {accuracy:.1f}%")

print()
print("------------------------------------------")
print("| Classe | Précision | Rappel | F1-score |")
print("------------------------------------------")
for i in range(num_classes):
  print(f"| {class_labels[i]:7s}|   {class_res[i]['Precision']:.2f}    |  {class_res[i]['Rappel']:.2f}  |   {class_res[i]['F-score']:.2f}   |")
  print("------------------------------------------")


En pratique, il est délicat de trouver une bonne combinaison des fonctions de perte tel que vous l'avez fait sur les cellules précédentes. L'entropie croisée et l'erreur quadratique moyenne donnent des valeurs trop différentes pour être combinables efficacement.

Une variante, peut-être plus simple à faire fonctionner, est d'utiliser uniquement l'erreur quadratique moyenne comme perte pour toutes les sorties. C'est cette variante qui est implémentée dans l'algorithme YOLO, dont nous implémenterons une version simplifiée dans le prochain TP.
Testez cette solution ci-dessous. Comme sur l'exercice précédent, n'hésitez pas à faire varier le poids des différents éléments de la fonction de coût.

In [ ]:
batch_size=16
model = create_model_localisation()
opt = Adam(learning_rate=3e-4)

loss = ['binary_crossentropy', 'mean_squared_error', 'categorical_crossentropy']
metrics =[ ['accuracy'], [iou()], ['accuracy']]
loss_weights = [1, 1, 1]

model.compile(loss=loss,
              optimizer=opt,
              metrics=metrics,
              loss_weights=loss_weights
              )
history = model.fit(x_train, [y_train[:,0], y_train[:,1:5], y_train[:,5:]],
              epochs=50,
              batch_size=batch_size,
              validation_data=(x_val, [y_val[:,0], y_val[:,1:5], y_val[:,5:]]))


In [ ]:
# Analyse des résultats : courbes d'évolution de la fonction de perte, et de l'IoU des boîtes englobantes ainsi que de la précision des classes prédites
plot_training_analysis(history, metric='loss')
plot_training_analysis(history, metric='coord_IoU')
plot_training_analysis(history, metric='classes_accuracy')

# Prédiction des données de test
y_pred_presence, y_pred_coords, y_pred_classes = model.predict(x_test)
y_pred = np.zeros(y_test.shape)
for i in range(y_pred.shape[0]):
  y_pred[i, 0] = y_pred_presence[i]
  y_pred[i, 1:5] = y_pred_coords[i]
  y_pred[i, 5:] = y_pred_classes[i]

# Affichage des résultats sur plusieurs images
print_data_localisation(x_test, y_test, y_pred = y_pred, id=1, image_size=IMAGE_SIZE)
print_data_localisation(x_test, y_test, y_pred = y_pred, id=2, image_size=IMAGE_SIZE)
print_data_localisation(x_test, y_test, y_pred = y_pred, id=3, image_size=IMAGE_SIZE)
print_data_localisation(x_test, y_test, y_pred = y_pred, id=4, image_size=IMAGE_SIZE)
print_data_localisation(x_test, y_test, y_pred = y_pred, id=5, image_size=IMAGE_SIZE)
print_data_localisation(x_test, y_test, y_pred = y_pred, id=6, image_size=IMAGE_SIZE)
print_data_localisation(x_test, y_test, y_pred = y_pred, id=7, image_size=IMAGE_SIZE)
print_data_localisation(x_test, y_test, y_pred = y_pred, id=8, image_size=IMAGE_SIZE)


class_res, accuracy = global_accuracy(y_test, y_pred, iou_thres=0.4)

print(f"La précision globale est de {accuracy:.1f}%")

print()
print("------------------------------------------")
print("| Classe | Précision | Rappel | F1-score |")
print("------------------------------------------")
for i in range(num_classes):
  print(f"| {class_labels[i]:7s}|   {class_res[i]['Precision']:.2f}    |  {class_res[i]['Rappel']:.2f}  |   {class_res[i]['F-score']:.2f}   |")
  print("------------------------------------------")


Compte-tenu de la taille réduite de la base de données, les résultats ne sont pas mal du tout ! On observe quelques confusions entre certaines classes mais les prédictions sont souvent intéressantes.

Il devrait cependant subsister un fort surapprentissage à ce stade. Comme nous l'avons vu dans de précédents TPs, vous avez plusieurs possibilités qui s'offrent à vous pour le corriger :

*   Augmentation de la base de données. Vous pouvez pour cela vous appuyer sur l'exemple fourni en TP précédent, avec une classe *Sequence* et l'utilisation de la librairie *Albumentation*. Je vous fournis une Sequence qui fonctionne ci-dessous.
*   Utilisation de *transfer learning* : partant d'un réseau entraîné sur ImageNet (qui contient de nombreuses classes d'animaux), vous bénéficieriez de filtres très généraux qui aiderait à limiter le surapprentissage.



# Code fourni pour vous aider à mettre en place l'augmentation de données

Définition des augmentations

In [ ]:
import cv2 as cv
from albumentations import (Compose, RandomBrightnessContrast, HorizontalFlip)
import albumentations as A

AUGMENTATIONS_TRAIN = Compose([
    HorizontalFlip(),
    RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

Une sequence qui permet d'implanter l'augmentation de données :

In [ ]:
from tensorflow.keras.utils import Sequence

class MangeoireSequence(Sequence):
    # Initialisation de la séquence avec différents paramètres
    def __init__(self, x_set, y_set, batch_size,augmentations):
        self.x, self.y = x_set, y_set
        self.classes = ['MESCHA', 'VEREUR', 'ECUROU', 'PIEBAV', 'SITTOR', 'PINARB', 'MESNOI', 'MESNON', 'MESBLE', 'ROUGOR', 'ACCMOU']
        self.batch_size = batch_size
        self.augment = augmentations
        self.indices1 = np.arange(x_set.shape[0])
        np.random.shuffle(self.indices1) # Les indices permettent d'accéder
        # aux données et sont randomisés à chaque epoch pour varier la composition
        # des batches au cours de l'entraînement

    # Fonction calculant le nombre de pas de descente du gradient par epoch
    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    # Il y a des problèmes d'arrondi dans les conversions de boîtes englobantes
    # internes à la librairie Albumentations
    # Pour les contourner, si les boîtes sont trop proches des bords, on les érode un peu
    def erode_bounding_box(self, box):
        epsilon = 0.01

        xmin = max(box[0] - box[2]/2, epsilon)
        ymin = max(box[1] - box[3]/2, epsilon)
        xmax = min(box[0] + box[2]/2, 1-epsilon)
        ymax = min(box[1] + box[3]/2, 1-epsilon)

        cx = xmin + (xmax - xmin)/2
        cy = ymin + (ymax - ymin)/2
        width = xmax - xmin
        height = ymax - ymin

        return np.array([cx, cy, width, height])

    # Application de l'augmentation de données à chaque image du batch et aux
    # boîtes englobantes associées
    def apply_augmentation(self, bx, by):

        batch_x = np.zeros((bx.shape[0], IMAGE_SIZE, IMAGE_SIZE, 3))
        batch_y = by

        # Pour chaque image du batch
        for i in range(len(bx)):
            bboxes = []
            box = by[i,1:5]
            # Dénormalisation des coordonnées de boites englobantes
            box[0] = (box[0]*y_std[1] + y_mean[1])
            box[1] = (box[1]*y_std[2] + y_mean[2])
            box[2] = (box[2]*y_std[3] + y_mean[3])
            box[3] = (box[3]*y_std[4] + y_mean[4])
            box = self.erode_bounding_box(box)
            bboxes.append(box)

            class_labels = []
            class_id = np.argmax(by[i, 5:])
            class_labels.append(self.classes[class_id])

            img = bx[i].astype('float32')

            # Application de l'augmentation à l'image et aux masques
            transformed = self.augment(image=img.astype('float32'), bboxes=bboxes, class_labels=class_labels)
            batch_x[i] = transformed['image']
            batch_y_transformed = transformed['bboxes']

            # Renormalisation des coordonnées de boîte englobante transformée
            batch_y[i, 1] = (batch_y_transformed[0][0] - y_mean[1])/y_std[1]
            batch_y[i, 2] = (batch_y_transformed[0][1] - y_mean[2])/y_std[2]
            batch_y[i, 3] = (batch_y_transformed[0][2] - y_mean[3])/y_std[3]
            batch_y[i, 4] = (batch_y_transformed[0][3] - y_mean[4])/y_std[4]

        return batch_x, batch_y

    # Fonction appelée à chaque nouveau batch : sélection et augmentation des données
    def __getitem__(self, idx):
        batch_x = self.x[self.indices1[idx * self.batch_size:(idx + 1) * self.batch_size]]
        batch_y = self.y[self.indices1[idx * self.batch_size:(idx + 1) * self.batch_size]]
        batch_x, batch_y = self.apply_augmentation(batch_x, batch_y)

        batch_y = np.array(batch_y)
        return np.array(batch_x), [batch_y[:,0], batch_y[:,1:5], batch_y[:,5:]]

    # Fonction appelée à la fin d'un epoch ; on randomise les indices d'accès aux données
    def on_epoch_end(self):
        np.random.shuffle(self.indices1)



In [ ]:
# Instanciation d'une Sequence
train_gen = MangeoireSequence(x_train, y_train, 16, augmentations=AUGMENTATIONS_TRAIN)

# Pour tester la séquence, nous sélectionnons les éléments du premier batch et les affichons
batch_x, batch_y = train_gen.__getitem__(0)

y_batch = np.zeros((batch_y[0].shape[0],1+4+num_classes))

for i in range(batch_y[0].shape[0]):
  y_batch[i, 0] = batch_y[0][i]
  y_batch[i, 1:5] = batch_y[1][i]
  y_batch[i, 5:] = batch_y[2][i]

print_data_localisation(batch_x, y_batch, image_size=IMAGE_SIZE)